필요한 도구 임포트와 입력의 정의

In [28]:
import torch
import torch.nn as nn

In [ ]:
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


합성곱층과 풀링 선언하기

In [30]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [31]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [32]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


구현체를 연결하여 모델 만들기

In [33]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [34]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [35]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [36]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [37]:
out.size(0)

1

In [38]:
out.size(1)

64

In [39]:
out.size(2)

7

In [40]:
out.size(3)

7

In [41]:
print(out)

tensor([[[[ 1.3913e+32,  4.1255e+32,  1.1801e+34,  ...,  1.1238e+33,
            1.2028e+33,  2.1065e+33],
          [ 5.8696e+32,  1.4245e+32,  4.3493e+34,  ...,  2.9707e+33,
            1.0887e+34,  5.6206e+33],
          [ 1.1858e+33,  2.3237e+33,  3.3012e+33,  ...,  1.3912e+31,
            2.1535e+33,  1.2335e+33],
          ...,
          [ 4.3878e+33,  1.3819e+34,  1.5421e+33,  ...,  2.1541e+32,
            2.4563e+33,  1.4217e+33],
          [ 5.5860e+31,  8.0321e+32,  1.8014e+33,  ...,  1.0385e+33,
            2.6585e+33,  2.6032e+33],
          [ 2.2742e+30,  2.2298e+33,  1.0003e+33,  ...,  5.3565e+32,
            1.2636e+33,  4.1871e+32]],

         [[ 4.7463e+32,  1.5900e+32,  3.5813e+34,  ...,  5.1589e+33,
            8.9914e+33,  1.6922e+33],
          [ 1.9709e+32,  1.5041e+32,  9.7043e+32,  ...,  9.3410e+31,
            7.7399e+33,  9.6623e+31],
          [ 9.0513e+33,  1.6847e+33,  1.5958e+33,  ...,  4.8002e+33,
            3.3400e+33,  2.1546e+33],
          ...,
     

In [42]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [43]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


CNN으로 MNIST 분류하기 이제 찐찐!!

In [44]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [46]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [47]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [48]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [49]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [50]:
model = CNN().to(device)

In [51]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [52]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [53]:
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225560248
[Epoch:    2] cost = 0.0630551875
[Epoch:    3] cost = 0.0462681577
[Epoch:    4] cost = 0.0374339223
[Epoch:    5] cost = 0.0314037167
[Epoch:    6] cost = 0.0261414386
[Epoch:    7] cost = 0.0216277987
[Epoch:    8] cost = 0.0179725569
[Epoch:    9] cost = 0.0158094056
[Epoch:   10] cost = 0.0131764095
[Epoch:   11] cost = 0.00995210093
[Epoch:   12] cost = 0.00957405847
[Epoch:   13] cost = 0.00827191118
[Epoch:   14] cost = 0.00659538386
[Epoch:   15] cost = 0.00589284115


In [54]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)

    correct_prediction = torch.argmax(prediction, 1) == Y_test

    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())


/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9883999824523926
